In [226]:
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

In [227]:
df = pd.read_csv('data_treated.csv')
df

,Run,Wcat [g],Temp [degC],VolF [mL/min],molF [mmol/min],xNH3,xN2,xH2,Conv,molF_NH3,...,xNH3_cal,xN2_cal,xH2_cal,Fin [mmol/min],Fout [mmol/min],Vin [mL/min],Vout [mL/min],Vavg [mL/min],Vcat [mL],rt [min]
0,0,0.3,300,30,1.226182,0.925,0.019,0.056,0.038961,1.178409,...,0.925,0.01875,0.05625,1.226182,1.273955,57.670636,59.917543,58.786933,0.402685,0.005586
1,1,0.3,300,60,2.452364,0.948,0.013,0.039,0.026694,2.386900,...,0.948,0.01300,0.03900,2.452364,2.517827,115.341271,118.420196,116.873975,0.402685,0.002793
2,2,0.3,300,90,3.678546,0.951,0.012,0.036,0.025115,3.586158,...,0.951,0.01225,0.03675,3.678546,3.770934,173.011907,177.357157,175.175550,0.402685,0.001862
3,3,0.3,350,30,1.226182,0.795,0.051,0.154,0.114206,1.086144,...,0.795,0.05125,0.15375,1.226182,1.366219,62.701660,69.862574,66.217597,0.402685,0.005138
4,4,0.3,350,60,2.452364,0.843,0.039,0.118,0.085187,2.243454,...,0.843,0.03925,0.11775,2.452364,2.661274,125.403320,136.086078,130.671929,0.402685,0.002569
5,5,0.3,350,90,3.678546,0.875,0.031,0.094,0.066667,3.433310,...,0.875,0.03125,0.09375,3.678546,3.923782,188.104981,200.645313,194.307707,0.402685,0.001713
6,6,0.3,400,30,1.226182,0.514,0.122,0.365,0.321004,0.832573,...,0.514,0.12150,0.36450,1.226182,1.619791,67.732685,89.475145,78.100155,0.402685,0.004756
7,7,0.3,400,60,2.452364,0.647,0.088,0.265,0.214329,1.926751,...,0.647,0.08825,0.26475,2.452364,2.977977,135.465370,164.499538,149.512899,0.402685,0.002378
8,8,0.3,400,90,3.678546,0.713,0.072,0.216,0.167542,3.062234,...,0.713,0.07175,0.21525,3.678546,4.294858,203.198055,237.242329,219.780910,0.402685,0.001585
9,9,0.3,450,30,1.226182,0.202,0.200,0.599,0.663894,0.412128,...,0.202,0.19950,0.59850,1.226182,2.040236,72.763710,121.071064,94.876506,0.402685,0.004427


In [228]:
df_rx = pd.DataFrame()
df_rx['Temp [K]'] = df['Temp [degC]'] + 273.15
df_rx['F0 [mmol/min]'] = df['Fin [mmol/min]']
df_rx['F [mmol/min]'] = df['Fin [mmol/min]']*(1-df['Conv'])
df_rx['rt [min]'] = df['rt [min]']

df_rx

,Temp [K],F0 [mmol/min],F [mmol/min],rt [min]
0,573.15,1.226182,1.178409,0.005586
1,573.15,2.452364,2.386900,0.002793
2,573.15,3.678546,3.586158,0.001862
3,623.15,1.226182,1.086144,0.005138
4,623.15,2.452364,2.243454,0.002569
5,623.15,3.678546,3.433310,0.001713
6,673.15,1.226182,0.832573,0.004756
7,673.15,2.452364,1.926751,0.002378
8,673.15,3.678546,3.062234,0.001585
9,723.15,1.226182,0.412128,0.004427


In [229]:
kp = np.load('best_x_preExp.npy')

In [230]:
class NH3_decomposition:
    def __init__(self, data_df,kp):
        self.data = data_df.copy()
        self.T = df_rx['Temp [K]'].values
        self.P = 1 # atm

        self.RT = self.data['rt [min]'].values
        self.F0 = self.data['F0 [mmol/min]'].values
        self.nNH3 = self.data['F [mmol/min]'].values
        self.kp = kp

    def partial_pressure(self, N, N0):
        nNH3 = N
        nN2 = 0.5*(N0-N)
        nH2 = 1.5*(N0-N)
        pNH3 = self.P*nNH3/(nNH3+nN2+nH2)
        pN2 = self.P*nN2/(nNH3+nN2+nH2)
        pH2 = self.P*nH2/(nNH3+nN2+nH2)

        return np.array([pNH3, pN2, pH2])
    
    def rate_equation(self, N, E, N0, T):
        R = 8.314 # J/mol/K
        T0 = 450+273.15
        
        k0 = (10**self.kp[0])*np.exp(-E[0]/R*(1/T-1/T0))
        k1 = (10**self.kp[1])*np.exp(E[1]/R*(1/T-1/T0))
        k2 = (10**self.kp[2])*np.exp(E[2]/R*(1/T-1/T0))
        k3 = (10**self.kp[3])*np.exp(E[3]/R*(1/T-1/T0))

        p = self.partial_pressure(N, N0)
        theta = 1 + k1*p[0] + (p[1]/k2)**0.5 + (p[2]/k3)**0.5
        r = k0*k1*p[0]/theta**4
        return r

    def ode(self, t, y, E, N0, T):
        N = y[0]

        rNH3 = self.rate_equation(N, E, N0, T)

        dNH3dt = -rNH3*0.3

        return [dNH3dt]

    def solve_ode(self, E, N0, tend, T):
        N0 = [N0]
        t_span = (0, tend)
        sol = solve_ivp(
            fun = lambda t, y: self.ode(t, y, E, N0, T),
            t_span = t_span,
            y0 = N0,
            t_eval = [tend],
            method = 'LSODA'
        )
        return sol.y[0]

    def obj_fun(self, params):

        E = 10**params
        err = 0

        for i in range(len(self.RT)):
            N0 = self.F0[i]
            tend = self.RT[i]
            T = self.T[i]
            N_pred = self.solve_ode(E, N0, tend, T)

            err += np.sum((self.nNH3[i] - N_pred[0])**2)
        return err

    def est_params(self):

        x0 = np.ones(4)
        best_x = np.ones(4)
        best_loss = float('inf')

        for i in range(100):
            res = minimize(
                fun = self.obj_fun,
                x0 = x0,
                method = 'L-BFGS-B',
                bounds = [(-7, 7) for _ in range(len(x0))],
                options = {'maxiter':1}
            )
            # 현재 시도의 loss 출력
            print(f'시도 {i+1} - x0: {x0}, Loss: {res.fun:.6f}, Best Loss: {best_loss:.6f}')

            if res.fun < best_loss:
                best_loss = res.fun
                best_x = res.x.copy()
                x0 = res.x.copy()
            else:
                x0 = best_x.copy()

            # x0에 랜덤 변화 추가
            x0 = x0 * (1 + np.random.randn(len(x0)) * 0.5)
            
        return best_x

In [231]:
sim = NH3_decomposition(df_rx,kp)
best_x = sim.est_params()

시도 1 - x0: [1. 1. 1. 1.], Loss: 13.604251, Best Loss: inf
시도 2 - x0: [0.73932205 1.35846584 0.79674136 1.04804116], Loss: 13.608559, Best Loss: 13.604251
시도 3 - x0: [1.44538196 1.34501695 0.22897008 0.47444873], Loss: 13.599816, Best Loss: 13.604251
시도 4 - x0: [1.67861715 1.40058005 0.1463834  0.75301539], Loss: 13.596408, Best Loss: 13.599816
시도 5 - x0: [1.40860463 1.15494134 0.24060936 0.79071383], Loss: 13.599834, Best Loss: 13.596408
시도 6 - x0: [1.94536475 1.78353758 0.0588811  0.85244592], Loss: 13.593263, Best Loss: 13.596408
시도 7 - x0: [2.71518424 1.46991566 0.05576788 0.84738125], Loss: 13.381893, Best Loss: 13.593263
시도 8 - x0: [ 2.77139891  2.55516059 -0.02566358  0.78097891], Loss: 13.372280, Best Loss: 13.381893
시도 9 - x0: [ 0.25583362  1.82840212 -0.03464189  0.549286  ], Loss: 13.615776, Best Loss: 13.372280
시도 10 - x0: [ 2.83573517  1.66574971 -0.0336554   0.75620823], Loss: 13.246303, Best Loss: 13.372280
시도 11 - x0: [-0.30614851  0.07210019 -0.04577286  0.42035636], Lo

In [234]:
np.save('best_x_E.npy', best_x)